# Ejercicio
Predecir el precio de un piso en función de la superficie construida y el distrito

In [1]:
#!pip install --upgrade typing-extensions

In [2]:
#!pip install gradio

In [62]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [63]:
df = pd.read_csv('pisos_con_habitaciones_banos.csv')
df.head()

,superficie_construida,precio,distritos_id,dormitorios,banos
0,600,5400000,10,5,4.5
1,600,3800000,10,5,4.5
2,59,330000,9,1,1.0
3,63,402000,8,1,1.0
4,300,1750000,10,5,4.5


## EDA (Exploratory Data Analisis)

In [64]:
# Cantidad de nulos de cada variable
df.isnull().sum()

superficie_construida     0
precio                    0
distritos_id              0
dormitorios               0
banos                    17
dtype: int64

In [66]:
df[df['banos'].isnull()]

,superficie_construida,precio,distritos_id,dormitorios,banos
40577,262,686000,9,2,NaN
40578,262,954000,9,2,NaN
61167,120,825300,11,3,NaN
61168,164,910300,11,3,NaN
61169,136,652300,11,4,NaN
61170,136,665300,11,4,NaN
61171,120,890300,11,4,NaN
61172,194,985300,11,4,NaN
61173,193,880300,11,5,NaN
61174,193,895300,11,5,NaN


Esos valores nulos se van a imputar con la mediana:

In [69]:
# Calcular la mediana de la columna 'banos'
mediana_banos = df['banos'].median()

# Rellenar los valores nulos con la mediana
df['banos'].fillna(mediana_banos, inplace = True)

In [70]:
# Cantidad de nulos de cada variable
df.isnull().sum()

superficie_construida    0
precio                   0
distritos_id             0
dormitorios              0
banos                    0
dtype: int64

In [71]:
# Valores únicos de distritos_id
df['distritos_id'].unique()

array([10,  9,  8,  7,  4, 17, 12, 14, 35, 15, 11, 16, 21,  1, 13, 22, 20,
       18, 24, 25])

In [72]:
# Información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67100 entries, 0 to 67099
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   superficie_construida  67100 non-null  int64  
 1   precio                 67100 non-null  int64  
 2   distritos_id           67100 non-null  int64  
 3   dormitorios            67100 non-null  int64  
 4   banos                  67100 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 2.6 MB


In [73]:
# X son las características (superficie_construida y distritos_id) e Y es lo que queremos predecir (precio) 
x = df[['superficie_construida', 'distritos_id', 'dormitorios', 'banos']]
y = df['precio']

Creamos visualizaciónes para facilitar una exploración manual. Primero preparamos nuestros dataframes, lo dividimos en dos: un representará las características y el otro los precios:

In [74]:
# Correlación de Pearson
correlacion_superficie_precio = df['superficie_construida'].corr(df['precio']) 
print('------- Correlación de Pearson m2 vs precio ------') 
print(correlacion_superficie_precio)

correlacion_dormitorios_precio = df['dormitorios'].corr(df['precio']) 
print('------- Correlación de Pearson dormitorios vs precio ------') 
print(correlacion_dormitorios_precio)

correlacion_banos_precio = df['banos'].corr(df['precio']) 
print('------- Correlación de Pearson banos vs precio ------') 
print(correlacion_banos_precio)

------- Correlación de Pearson m2 vs precio ------
0.16576537534880018
------- Correlación de Pearson dormitorios vs precio ------
0.0688522419652047
------- Correlación de Pearson banos vs precio ------
0.1323728158042718


## Modelo LinearRegression

### Paso 1: Lectura de datos

In [75]:
df_distritos = pd.read_csv('distritos.csv') # Nombres e ids de distrito
df_distritos.head()

,id,distrito
0,1,ARGANZUELA
1,4,CENTRO
2,7,SALAMANCA
3,8,CHAMARTIN
4,9,TETUAN


### Paso2: Prepración de variables

In [76]:
# X son las características (superficie_construida, dormitorios, banos y distritos_id) e Y es lo que queremos predecir (precio) 
x = df[['superficie_construida', 'distritos_id', 'dormitorios', 'banos']]
y = df['precio']

In [77]:
# Se añade una nueva columna al DataFrame que representa el precio por metro cuadrado

df['precio_m2'] = df['precio'] / df['superficie_construida']

### Paso 3: Limpieza de datos

Dado que ya hemos realizado el EDA, hemos visto que hay algunos valores atípicos que deforman bastante el gráfico. 
Se filtran los valores atípicos del precio por metro cuadrado usando los percentiles 1 y 99. Esto ayuda a mejorar la calidad de los datos eliminando los extremos.

In [78]:
# Eliminar valores atípicos usando el percentil 99 y 1 para precio m2, dormitorios y banos
df = df[(df['precio_m2'] < df['precio_m2'].quantile(0.9)) & (df['precio_m2'] > df['precio_m2'].quantile(0.1))]
df = df[(df['dormitorios'] < df['dormitorios'].quantile(0.9)) & (df['dormitorios'] > df['dormitorios'].quantile(0.1))]
df = df[(df['banos'] < df['banos'].quantile(0.9)) & (df['banos'] > df['banos'].quantile(0.1))]

In [79]:
# Con los datos limpios volvemos a reasignar x e y 
x = df[['superficie_construida', 'distritos_id', 'dormitorios', 'banos']] 
y = df['precio']

### Paso 4: Normalización de Datos

**Normalización de la Superficie Construida, Dormitorios y Banos**
- Se normalizan estas variables restando la media y dividiendo por la desviación estándar. Esto coloca los datos en una escala común sin distorsionar las diferencias en los rangos de valores.

**Creación de Características Normalizadas**
- Se crea un nuevo DataFrame (caracteristicas) que combina 'superficie_construida' normalizada y 'distritos_id’.

**Normalización del Precio**
- Se normaliza el precio de la misma manera que las otras variables

In [80]:
# Normalizar los datos
media_superficie = x['superficie_construida'].mean()
std_superficie = x['superficie_construida'].std()
superficie_normalizada = (df['superficie_construida'] - media_superficie) / std_superficie

media_dormitorios = x['dormitorios'].mean()
std_dormitorios = x['dormitorios'].std()
dormitorios_normalizados = (df['dormitorios'] - media_dormitorios) / std_dormitorios

media_banos = x['banos'].mean()
std_banos = x['banos'].std()
banos_normalizados = (df['banos'] - media_banos) / std_banos

In [81]:
# Combinar las variables normalizadas con 'distritos_id' no normalizado
caracteristicas = pd.DataFrame({
    'superficie_construida': superficie_normalizada,
    'dormitorios': dormitorios_normalizados,
    'banos': banos_normalizados,
    'distritos_id': x['distritos_id']
})

In [82]:
precios = (y - y.mean()) / y.std()

### Paso 5: División de Datos en Entrenamiento y Prueba

Se divide el conjunto de datos en un conjunto de entrenamiento y uno de prueba. Esto es muy importante para poder evaluar el modelo de manera efectiva y evitar el overfitting.

In [83]:
# Dividir los datos en conjunto de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(caracteristicas, precios, test_size=0.2, random_state=42)

### Paso 6: Creación y entrenamiento del modelo

In [84]:
# Crear el modelo de regresión lineal
model = LinearRegression()

In [85]:
# Entrenar el modelo
model.fit(x_train, y_train)

LinearRegression()

### Paso 7: Evaluación del Modelo

In [86]:
# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(x_test)

In [87]:
# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred) 
r2 = r2_score(y_test, y_pred)

In [88]:
print(f'MSE: {mse}') 
print(f'R²: {r2}')

MSE: 0.36955070314637
R²: 0.6252384327491572


**Interpretación:**

- El MSE es 0.3696, lo que significa que, en promedio, las predicciones del modelo difieren en aproximadamente 0.3696 unidades cuadradas de la verdadera etiqueta.

- Un R² de 0.6252 significa que aproximadamente el 62.52% de la variabilidad en los datos de respuesta es explicada por el modelo de regresión lineal.

### Paso 8a: Implementación Práctica I

**Mapeo de Distritos**

Se crea un diccionario que mapea los identificadores de distrito a sus nombres. El objetivo es hacerlo más amigable al usuario en el interfaz que vamos a preparar.

In [89]:
distritos = dict(zip(df_distritos['id'],df_distritos['distrito']))

In [90]:
# Crear la lista de opciones para el Dropdown como tuplas (nombre, id) 
opciones_distritos = [(nombre, id) for id, nombre in distritos.items()]

### Paso 8b: Implementación Práctica II

**Función de Predicción Personalizada**

Se define una función predict_precio que normaliza la superficie construida de una entrada, la combina con el identificador de distrito y utiliza el modelo para hacer una predicción del precio.

In [92]:
def predict_precio (superficie, dormitorios, banos, distrito):
    #Normalizar solo la superficie construida
    superficie_normalizada = (superficie - media_superficie) / std_superficie
    dormitorios_normalizados = (dormitorios - media_dormitorios) / std_dormitorios
    banos_normalizados = (banos - media_banos) / std_banos
    
    #Convertirlos en un DataFrame de Pandas para la predicción
    datos_prediccion = pd.DataFrame({
        'superficie_construida': [superficie_normalizada],
        'dormitorios': [dormitorios_normalizados],
        'banos': [banos_normalizados],
        'distritos_id': [distrito]
    })
    
    #Se hace la predicción (sin normalizar el distrito)
    precio_pred_normalizado = model.predict(datos_prediccion)
    
    #Se desnormaliza la predicción del precio para mostrarlo al usuario
    precio_pred = precio_pred_normalizado * y.std() + y.mean()
    
    precio_formateado = '{:20,d} €'.format(int(precio_pred[0]))
    return precio_formateado

### Paso 9: Interfaz de Usuario

**Creación de la Interfaz con Gradio**

Se configura una interfaz de usuario con Gradio, donde los usuarios pueden introducir la superficie y seleccionar un distrito para obtener una predicción del precio del inmueble.
Se lanza la interfaz para que los usuarios interactúen con el modelo.

In [96]:
import gradio as gr

In [97]:
print(gr.__version__)

4.15.0


In [98]:
# Crear la interfaz con Gradio
iface = gr.Interface(
    fn=predict_precio,
    inputs=[
        gr.Number(label="Superficie (m²)"),
        gr.Number(label="Dormitorios"),
        gr.Number(label="Baños"),
        gr.Dropdown(choices=opciones_distritos, label="Distrito")
    ],
    outputs="text",  # Cambiado a 'text' para permitir una cadena
    title="Predicción de Precio de Inmueble",
    description="Introduce la superficie, dormitorios, baños y selecciona el distrito para predecir el precio"
)


In [99]:
# Ejecutar la interfaz
iface.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
